In [1]:
#ΦΟΙΤΗΤΕΣ ΕΠΙΜΕΛΕΙΑ:
#ΦΟΙΤΗΤΗΣ1 : Βόικος Στέφανος (03118162)
#
#ΦΟΙΤΗΤΗΣ2 : Μιχαλίτσης Αλκιβιάδης (03118868)

from google.colab import drive
drive.mount('/gdrive')
import os
os.listdir('/gdrive/My Drive')
path='/gdrive/My Drive/'

Mounted at /gdrive


In [2]:
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog
#install pyswip
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.18).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.downl

In [3]:
#Θεμα 1ο
import pandas as pd
from pyswip import Prolog

# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv(path + "movies_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,production_companies,production_countries,release_date,gross,duration,spoken_languages,status,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,880674609,148,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,A Plan No One Escapes,Spectre,6.3,4466,2015,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,1084939099,165,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,2012,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,284139100,132,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,2012,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


In [4]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

#create World
#Ορίζουμε τον κόσμο μας
from pyswip import Prolog
prolog = Prolog()

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
for row in data.itertuples(index=True, name='Pandas'):
  movie_title = clean_text(getattr(row, 'movie_title'))

  for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ movie_title +"','"+ genre +"')")
    
  for director in clean_text(getattr(row, 'director_name')).split("|"):
    literals.append("director('"+ movie_title +"','"+ director +"')")

  colorful = True  
  for plot in clean_text(getattr(row, 'plot_keywords')).split("|"):
    literals.append("plot_key('"+ movie_title +"','"+ plot +"')")
    if plot == 'black and white':
      colorful = False
  if colorful == True: 
    #literals.append("with_color('"+ movie_title +"','"+ plot +"')")
    literals.append("with_color('"+ movie_title +"')")

  for star in clean_text(getattr(row, 'actor_1_name')).split("|"):
    literals.append("plot_key('"+ movie_title +"','"+ star +"')")

  for actor in clean_text(getattr(row, 'actor_1_name')).split("|"):
    literals.append("actor('"+ movie_title +"','"+ actor +"')")
    
  for actor in clean_text(getattr(row, 'actor_2_name')).split("|"):
    literals.append("actor('"+ movie_title +"','"+ actor +"')")
    
  for actor in clean_text(getattr(row, 'actor_3_name')).split("|"):
    literals.append("actor('"+ movie_title +"','"+ actor +"')")
    
  for lang in clean_text(getattr(row, 'language')).split("|"):
    literals.append("language('"+ movie_title +"','"+ lang +"')")
  """
  for pop in clean_text(getattr(row, 'popularity')).split("|"):
    literals.append("popularity('"+ movie_title +"','"+ pop +"')")
    """
  vote = (getattr(row, 'vote_average'))
  if vote != "UNK":
    st = ("vote_average('"+ movie_title +"','"+ "{v:.1f}" +"')")
    #print(st.format(v = vote))
    #st = ("vote_average('"+ movie_title +"','{})")
    literals.append(st.format(v = vote))

  dur = (getattr(row, 'duration'))
  if dur != "UNK":
    st = ("duration('"+ movie_title +"','"+ "{d:.1f}" +"')")
    literals.append(st.format(d = dur))

#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
  prolog.assertz(literal)

#Επίσης μπορούμε να κάνουμε consult ένα έτοιμο αρχείο στον κόσμο όπως παρακάτω
prolog.consult(path +"db.pl")
#prolog.assertz('(directed_by(X,Y) :- findall(M,director(M,X),Y))')


#Θεμα 2ο

q1 = prolog.query("directed_by('Brian Herzlinger', Y)")
for soln in q1:
    s = soln['Y'] 
    print(s)
q1.close()

q2 = prolog.query("directed_by('Christopher Nolan', Y)")
for soln in q2:
    s = soln['Y'] 
    print(s)
q2.close()

[Atom('2228229')]
[Atom('846597'), Atom('1834117'), Atom('1851653'), Atom('1854981'), Atom('2134405'), Atom('836997'), Atom('837893'), Atom('1436165')]


In [6]:

prolog.assertz('''(find_sim_1(X, Y):-
                                        genre(X, G1),
                                        genre(Y, G1),
                                        X \= Y )''')

q = prolog.query("find_sim_1('Avatar', Y)")
s = set()
for soln in q:
  m = soln['Y'] 
  if m not in s:
    s.add(soln['Y'])
print(s)

{'Against the Wild', 'Mr. & Mrs. Smith', 'Gremlins', 'All Superheroes Must Die', 'The Island of Dr. Moreau', 'Saving Private Perez', 'Escape from New York', 'Silmido', 'I, Robot', '15 Minutes', 'Captain Phillips', 'Cutthroat Island', 'The Wizard of Oz', 'Torque', '55 Days at Peking', 'Alien Zone', 'Cypher', 'From Paris with Love', 'The Cat in the Hat', 'Love Me Tender', 'Money Talks', 'Kick-Ass', 'The Equalizer', 'V for Vendetta', 'Red Planet', 'The Phantom', 'Arthur and the Invisibles', 'Unnatural', 'Jackass Number Two', 'Pontypool', 'Do You Believe?', 'Attack the Block', 'Superman', 'The Last Stand', 'Independence Daysaster', 'The Sum of All Fears', 'High Heels and Low Lifes', 'Bangkok Dangerous', 'Volcano', 'Eight Below', 'Stardust', 'Daylight', 'The Prisoner of Zenda', 'I Spy', 'Moonraker', 'Boom Town', 'Deadpool', 'Solaris', 'Practical Magic', 'The Legend of Tarzan', '3 Days to Kill', 'Street Fighter: The Legend of Chun-Li', 'The Barbarians', 'Heroes of Dirt', 'Looper', 'Victor Fr

In [7]:
def simple_recommender(movie):
    s = set()
    q = prolog.query("find_sim_1('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
    
list(simple_recommender('Avatar'))[:5]

['Against the Wild',
 'Mr. & Mrs. Smith',
 'Gremlins',
 'All Superheroes Must Die',
 'The Island of Dr. Moreau']

In [8]:
#GenreRecommender
#Η συνάρτηση genre_recommender με είσοδο μία ταινία movie επιστρέφει τις πιο κοντινές στο είδος ταινίες με την movie (δηλαδή ταινίες με τουλάχιστον 3 κοινά genre-keywords με την movie). Αν οι ταινίες αυτές είναι λιγότερες από 10 τότε εξετάζει τις ταινίες σχετικά ίδιου είδους με την movie (ταινίες με τουλάχιστον 2 κοινά genre-keywords με την movie) κι τις επιστρέφει σε 2ο σύνολο.

#Recommends movies with similar genre 
def genre_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("very_common('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    if (len(answers) < 10):
      q = prolog.query("common('" + movie +"',M)")
      for soln in q:
        m = soln['M'] 
        if m not in s and m not in w:
          w.add(soln['M'])
      q.close()

    answers1 = s 
    answers2 = w 
    return answers1,answers2
def GenreRecommender(movie):
  (recom1, recom2) = list(genre_recommender(movie))
  recom1 = list(recom1)[:10]
  recom2 = list(recom2)[:10]
  print("Recommendation filter: GENRE")
  print("Chosen movie " + movie)
  print("Movies with same genre:")
  print(recom1)
  if(len(recom1) < 10):
    print("Movies with same genre:")
    print(recom2)
GenreRecommender('Avatar')

Recommendation filter: GENRE
Chosen movie Avatar
Movies with same genre:
['Krull', 'The Covenant', 'The Island', 'Puss in Boots', 'X-Men: First Class', 'The Matrix Revolutions', 'The Lord of the Rings: The Two Towers', 'Godzilla 2000', 'BloodRayne', '47 Ronin']


In [9]:
#ActorRecommender
#Η συνάρτηση actor_recommender με είσοδο μία ταινία movie επιστρέφει 10 ταινίες που έχουν τουλάχιστον 3 κοινούς ηθοποιούς με την movie, αλλά και τις ταινίες που έχουν τουλάχιστον 2 κοινούς ηθοποιούς με αυτήν.

#Recommends movies with common actors
def actor_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("all_actors('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    if (len(answers) < 10):
      q = prolog.query("many_actors('" + movie +"',M)")
      for soln in q:
        m = soln['M'] 
        if m not in s and m not in w:
          w.add(soln['M'])
      q.close()

    answers1 = s 
    answers2 = w 
    return answers1,answers2
def ActorRecommender(movie):
  (recom1, recom2) = list(actor_recommender(movie))
  recom1 = list(recom1)[:10]
  recom2 = list(recom2)[:10]
  print("Recommendation filter: ACTOR")
  print("Chosen movie " + movie)
  print("Movies with at least 3 actors in common:")
  print(recom1)
  if(len(recom1) < 10):
    print("Movies with some actors in common:")
    print(recom2)
ActorRecommender('The Talented Mr. Ripley')

Recommendation filter: ACTOR
Chosen movie The Talented Mr. Ripley
Movies with at least 3 actors in common:
[]
Movies with some actors in common:
['Contagion']


In [10]:
#Recommends movies of the same director
def director_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("common_director('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
def DirectorRecommender(movie):
  recom1 = list(director_recommender(movie))[:10]
  print("Recommendation filter: DIRECTOR")
  print("Chosen movie " + movie)
  print("Movies of the same director:")
  print(recom1)
DirectorRecommender('The Dark Knight Rises')

Recommendation filter: DIRECTOR
Chosen movie The Dark Knight Rises
Movies of the same director:
['Inception', 'The Dark Knight', 'Memento', 'The Prestige', 'Batman Begins', 'Insomnia', 'Interstellar']


In [11]:
#ColorRecommender
#Η συνάρτηση color_recommender με είσοδο μία ταινία movie επιστρέφει 10 ασπρόμαυρες ταινίες αν η movie είναι ασπρόμαυρη και 10 έγχρωμες διαφορετικά.

#Recommends movies of the same color
def color_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("common_color('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
def ColorRecommender(movie):
  recom1 = list(color_recommender(movie))[:10]
  print("Recommendation filter: COLOR")
  print("Chosen movie " + movie)
  print("Recommended movies:")
  print(recom1)
#example of black and white movie
ColorRecommender('The Phantom of the Opera')
print()
#example of movie with color
ColorRecommender('The Pianist')

Recommendation filter: COLOR
Chosen movie The Phantom of the Opera
Recommended movies:
['Frances Ha', 'Creative Control', 'The Giver', 'Renaissance', 'Frankenweenie']

Recommendation filter: COLOR
Chosen movie The Pianist
Recommended movies:
['Doubt', 'Against the Wild', 'Mr. & Mrs. Smith', 'Our Kind of Traitor', 'All Superheroes Must Die', 'Gremlins', 'The Island of Dr. Moreau', 'The Grudge', 'Kindergarten Cop', 'Saving Private Perez']


In [12]:
#BestRecommender
#Επιστρεφει True μήνυμα αν η επιλεγμένη ταινία έχει βαθμολογία μεγαλύτερη από 7 στο ΙΜDB αλλιώς False.

def BestRecommender(movie):
    q= prolog.query("vote_average('" + movie +"',Y)")
    #q = prolog.query("great_movie('" + movie +"')")
    #print (bool(list(q)))
    #q.close()
    for soln in q:
      s = soln['Y']
    #print(s)
    if s:
      #print(type (s[0]))
      if (float (s[0])) > 7.0:
       #print('Amazing movie: IMDb score greater than 8')
       return True
      else:
        #print('IMDb score lower than 8')
        return False
    q.close()
print(BestRecommender('The Godfather'))
#True
print(BestRecommender('Diamonds Are Forever'))
#False

True
False


In [13]:
#LanguageRecommender
#Η συνάρτηση language_recommender με είσοδο μία ταινία movie επιστρέφει 10 ταινίες με ίδια γλώσσα με τη movie.

#Recommends movies of the same language
def language_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("same_language('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
def LanguageRecommender(movie):
  recom1 = list(language_recommender(movie))[:10]
  print("Recommendation filter: LANGUAGE")
  print("Chosen movie " + movie)
  print("Recommended movies:")
  print(recom1)
LanguageRecommender('Mission: Impossible III')

Recommendation filter: LANGUAGE
Chosen movie Mission: Impossible III
Recommended movies:
['Run Lola Run', 'Miracle at St. Anna', 'Rollerball', 'X-Men: First Class', 'Krampus', 'The Train', 'Schindlers List', 'Reds', 'Inglourious Basterds', 'Enemy at the Gates']


In [14]:
#DurationRecommender
#Η συνάρτηση duration_recommender με είσοδο μία ταινία movie επιστρέφει 10 ταινίες με διάρκεια τουλάχιστον όση κα της movie.

#Recommends movies of the same language
def duration_recommender(movie):
    s = set()
    w = set()
    q = prolog.query("greater_duration('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s and m != movie:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
def DurationRecommender(movie):
  recom1 = list(duration_recommender(movie))[:10]
  print("Recommendation filter: DURATION")
  print("Chosen movie " + movie)
  print("Recommended movies:")
  print(recom1)
DurationRecommender('A Beautiful Mind')

Recommendation filter: DURATION
Chosen movie A Beautiful Mind
Recommended movies:
['The Tree of Life', 'Harry Potter and the Chamber of Secrets', 'Les Misérables', 'The Lord of the Rings: The Two Towers', 'Love in the Time of Cholera', 'Silmido', 'Im Not There.', 'Django Unchained', 'Topsy-Turvy', 'Pirates of the Caribbean: Dead Mans Chest']


In [15]:
prolog.assertz('''(find_sim_2(X, Y):-
                                        common(X, Y),
                                        some_actors(X, Y),
                                        X \= Y )''')
def Recommender2(movie):
    s = set()
    q = prolog.query("find_sim_2('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s:
            s.add(soln['M'])
    q.close()
    answers = s
    answers2 = set()
    #if movie has IMDB score > 7
    for mov in answers:
      if BestRecommender(mov):
        answers2.add(mov)
    return answers
list(Recommender2('Avatar'))[:10]


['The Book of Life',
 'Guardians of the Galaxy',
 'The Losers',
 'Star Trek Into Darkness',
 'Alien',
 'Crossroads']

In [16]:
prolog.assertz('''(find_sim_3(X, Y):-
                                        common(X, Y),
                                        common_plot(X, Y),
                                        X \= Y )''')
def Recommender3(movie):
    s = set()
    q = prolog.query("find_sim_3('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s:
            s.add(soln['M'])
    q.close()
    answers = s
    answers2 = set()
    #if movie has IMDB score > 7
    for mov in answers:
      if BestRecommender(mov):
        answers2.add(mov)
    return answers2
list(Recommender3('Avatar'))[:10]

['Interstellar']

In [17]:
prolog.assertz('''(find_sim_4(X, Y):-
                                        same_plot(X, Y),
                                        very_common(X, Y),
                                        same_language(X,Y),
                                        X \= Y )''')
def Recommender4(movie):
    s = set()
    q = prolog.query("find_sim_4('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
list(Recommender4('Avatar'))[:10]


['Pirates of the Caribbean: On Stranger Tides',
 'Star Trek Into Darkness',
 'The Fifth Element',
 'John Carter',
 'Predators']

In [18]:
prolog.assertz('''(find_sim_5(X, Y):-
                                        common_director(X, Y),
                                        very_common(X, Y),
                                        X \= Y )''')
def Recommender5(movie):
    s = set()
    q = prolog.query("find_sim_5('" + movie +"',M)")
    for soln in q:
        m = soln['M'] 
        if m not in s:
            s.add(soln['M'])
    q.close()
    answers = s
    return answers
list(Recommender5('Avatar'))[:10]

['The Abyss']

In [19]:
#GeneralRecommender
#Χρησιμοποιεί τους παραπάνω Recommenders κι εμφανίζει λίστα με τις όμοιες ταινίες σε φθίνουσα σειρά ομοιότητας. Ο χρήστης μπορεί να επιλέξει πόσες ταινίες θέλει να του προταθούν θέτοντας το δεύτερο όρισμα της συνάρτησης GeneralRecommender.

def GeneralRecommender(movie,x):
  answer = []
  temp = []
  answer = list(Recommender5(movie))[:x]
  if (len(answer) < x):
    temp = list(Recommender4(movie))
    for i in temp:
      if i not in answer:
        answer = answer + [i]
    answer = answer[:x]
  if (len(answer) < x):
    temp = list(Recommender3(movie))
    for i in temp:
      if i not in answer:
        answer = answer + [i]
    answer = answer[:x]
  if (len(answer) < x):
    temp = list(Recommender2(movie))
    for i in temp:
      if i not in answer:
        answer = answer + [i]
    answer = answer[:x]
  if (len(answer) < x):
    temp = list(simple_recommender(movie))
    for i in temp:
      if i not in answer:
        answer = answer + [i]
    answer = answer[:x]
  return answer
GeneralRecommender('Interstellar',10)

['Silent Running',
 'The Astronaut Farmer',
 'The Martian',
 'Metropolis',
 'Children of Men',
 'The Huntsman: Winters War',
 'The Other Side of Heaven',
 'Doubt',
 'Against the Wild',
 'Mr. & Mrs. Smith']

In [20]:
GeneralRecommender('Harry Potter and the Prisoner of Azkaban',20)

['Harry Potter and the Philosophers Stone',
 'Harry Potter and the Half-Blood Prince',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Order of the Phoenix',
 'The Tale of Despereaux',
 'Quest for Camelot',
 'The Fifth Element',
 'Against the Wild',
 'Gremlins',
 'The Island of Dr. Moreau',
 'Cutthroat Island',
 'The Wizard of Oz',
 'The Cat in the Hat',
 'Money Talks',
 'V for Vendetta',
 'The Phantom',
 'Arthur and the Invisibles',
 'Do You Believe?',
 'Superman']

In [21]:
#Θεμα 3ο
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score


#rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
#score_weights = {i:i + 1 for i in range(5)} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

rating_weights = {0: -1, 1: -0.3, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {i:i + 10 for i in range(5)} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

def train_recommender(ratings, rating_weights, score_weights, start=0, end = -1, movie_score = {}):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train,
    ενώ παράλληλα μπορούμε να αρχικοποιήσουμε και το movie_score το οποίο αποθηκεύονται τα score κάθε ταινίας.
    Έτσι αν έχουμε κάνει train τον recommender μας για τις πρώτες 10 ταινίες, μπορούμε να συνεχίσουμε για τις 
    υπόλοιπες χωρίς κάθε φορά να ξανά-διαμορφώνουμε το score για αυτές. Αυτό θα σας βοηθήσει στην εκτέλεση και χρονικά όταν ελέγχετε την 
    απόδοση σε υποσύνολα των ratings π.χ. πρώτες 10 ταινίες έπειτα τις επόμενες 20 κ.ο.κ.
    """
    if end == -1:
        end = len(ratings)

    ratings = ratings.iloc[range(start, end)]
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        similar_movies = Recommender5(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[4]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[4] 

        similar_movies = Recommender4(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[3]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[3]         
    
        similar_movies = Recommender3(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[2]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[2] 

        similar_movies = Recommender2(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[1]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[1] 

        similar_movies = simple_recommender(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[0]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[0] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 0
def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > 0)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics
train_ratings = pd.read_csv(path + "train_ratings.csv")
test_ratings = pd.read_csv(path + "test_ratings.csv")
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 0, 10)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

0it [00:00, ?it/s]

{'precision': 0.4921875, 'recall': 0.875, 'f1': 0.63}


In [22]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10, 20, movie_score)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

0it [00:00, ?it/s]

{'precision': 0.5111111111111111, 'recall': 0.9583333333333334, 'f1': 0.6666666666666666}


In [23]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 20, 30, movie_score)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

0it [00:00, ?it/s]

{'precision': 0.5147058823529411, 'recall': 0.9722222222222222, 'f1': 0.6730769230769229}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 30, 50, movie_score)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

0it [00:00, ?it/s]